In [3]:
import streamlit as st
from dotenv import load_dotenv, find_dotenv
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import BytesIO
import os

load_dotenv(find_dotenv())

True

In [4]:
blob_service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_STORAGE_CONNECTION_STRING"))


NameError: name 'os' is not defined